# 02-1 합성곱 신경망의 신호탄 AlexNet과 VGGNet

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/hm-dl/blob/main/02-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

## AlexNet

### 케라스로 AlexNet 만들기

In [1]:
import keras
from keras import layers

In [2]:
alexnet = keras.Sequential()
alexnet.add(layers.Conv2D(filters=96, kernel_size=11, strides=4,
                          activation='relu', input_shape=(227, 227, 3)))
alexnet.add(layers.MaxPooling2D(pool_size=3, strides=2))
alexnet.add(layers.Conv2D(filters=256, kernel_size=5, padding='same',
                          activation='relu'))
alexnet.add(layers.MaxPooling2D(pool_size=3, strides=2))
alexnet.add(layers.Conv2D(filters=384, kernel_size=3, padding='same',
                          activation='relu'))
alexnet.add(layers.Conv2D(filters=384, kernel_size=3, padding='same',
                          activation='relu'))
alexnet.add(layers.Conv2D(filters=256, kernel_size=3, padding='same',
                          activation='relu'))
alexnet.add(layers.MaxPooling2D(pool_size=3, strides=2))
alexnet.add(layers.Flatten())
alexnet.add(layers.Dense(4096, activation='relu'))
alexnet.add(layers.Dropout(0.5))
alexnet.add(layers.Dense(4096, activation='relu'))
alexnet.add(layers.Dropout(0.5))
alexnet.add(layers.Dense(1000, activation='softmax'))

### AlexNet 모델 구조 분석하기

In [3]:
alexnet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2  (None, 27, 27, 96)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 13, 13, 256)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 13, 13, 384)       885120    
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 384)       1

## VGGNet

### 케라스로 VGGNet 만들기

In [4]:
vggnet = keras.Sequential()
vggnet.add(layers.InputLayer(input_shape=(224, 224, 3)))

# 1, 2번째 블록
for n_filters in [64, 128]:
    for _ in range(2):
        vggnet.add(layers.Conv2D(filters=n_filters, kernel_size=3,
                                 padding='same', activation='relu'))
    vggnet.add(layers.MaxPooling2D(pool_size=2))

# 3, 4, 5번째 블록
for n_filters in [256, 512, 512]:
    for _ in range(3):
        vggnet.add(layers.Conv2D(filters=n_filters, kernel_size=3,
                                 padding='same', activation='relu'))
    vggnet.add(layers.MaxPooling2D(pool_size=2))

vggnet.add(layers.Flatten())
vggnet.add(layers.Dense(4096, activation='relu'))
vggnet.add(layers.Dense(4096, activation='relu'))
vggnet.add(layers.Dense(1000, activation='softmax'))

### VGGNet 모델 구조 분석하기

In [5]:
vggnet.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 224, 224, 64)      1792      
                                                                 
 conv2d_6 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 112, 112, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_8 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                 